<a href="https://colab.research.google.com/github/trieuduongle/VFIformer/blob/colab/VFIFormer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
!git clone https://github.com/trieuduongle/VFIformer.git '/content/drive/My Drive/VFIformer'

Cloning into '/content/drive/My Drive/VFIformer'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 122 (delta 68), reused 109 (delta 55), pack-reused 0
Receiving objects: 100% (122/122), 229.01 KiB | 5.45 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
%cd '/content/drive/My Drive/VFIformer'

/content/drive/My Drive/VFIformer


In [ ]:
!sudo apt-get update -y && sudo apt-get install python3.10

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3
!python --version
!sudo apt install python3-pip

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
  0            /usr/bin/python3.7    2         auto mode
* 1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.6    1         manual mode
  3            /usr/bin/python3.7    2         manual mode

Press <enter> to keep the current choice[*], or type selection number: 0
update-alternatives: using /usr/bin/python3.7 to provide /usr/bin/python3 (python3) in auto mode
Python 3.7.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [5]:
!git checkout colab && git pull && pip install -r requirements.txt -f https://download.pytorch.org/whl/cu111/torch_stable.html

Already on 'colab'
Your branch is up to date with 'origin/colab'.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/trieuduongle/VFIformer
   b1629e4..73c0627  colab      -> origin/colab
Updating b1629e4..73c0627
Fast-forward
 VFIFormer.ipynb | 471 ++++++++++++++++++++++++++++++++++++++++++++++++++++++--
 1 file changed, 454 insertions(+), 17 deletions(-)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     |████████████████████████████████| 11.2 MB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 54.8 MB/s 
     |████████████████████████████████| 125 kB 74.4 MB/s 
     |████████████████████████████████| 509 kB 74.4 MB/s 
     |█████████████▌               

In [ ]:
!pip uninstall -r requirements.txt

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --master_port=4175 train.py --launcher pytorch --gpu_ids 0 \
            --loss_l1 --loss_ter --loss_flow --use_tb_logger --batch_size 24 --net_name VFIformerSmall --name train_VFIformerSmall --max_iter 300 \
            --crop_size 192 --save_epoch_freq 5 --data_root "/content/drive/My Drive/datasets/vimeo_triplet"

In [35]:
!python test.py --data_root "/content/drive/My Drive/datasets/vimeo_triplet" --testset VimeoDataset --net_name VFIformerSmall \
  --resume "/content/drive/My Drive/VFIformer/weights/train_VFIformerSmall/snapshot/net_105.pth" \
  --save_result --save_folder "/content/drive/My Drive/datasets/vimeo_triplet/test_results"

Disabled distributed training.
2022-08-15 16:15:12,339 [INFO ]  Logging file is /content/drive/My Drive/datasets/vimeo_triplet/test_results/test_vfiformer/20220815_161512.log
2022-08-15 16:15:12,339 [INFO ]  random_seed:0
2022-08-15 16:15:12,340 [INFO ]  name:test_vfiformer
2022-08-15 16:15:12,340 [INFO ]  phase:test
2022-08-15 16:15:12,340 [INFO ]  gpu_ids:[0]
2022-08-15 16:15:12,340 [INFO ]  launcher:none
2022-08-15 16:15:12,340 [INFO ]  local_rank:0
2022-08-15 16:15:12,340 [INFO ]  net_name:VFIformerSmall
2022-08-15 16:15:12,340 [INFO ]  data_root:/content/drive/My Drive/datasets/vimeo_triplet
2022-08-15 16:15:12,340 [INFO ]  trainset:VimeoDataset
2022-08-15 16:15:12,341 [INFO ]  testset:VimeoDataset
2022-08-15 16:15:12,341 [INFO ]  crop_size:192
2022-08-15 16:15:12,341 [INFO ]  batch_size:1
2022-08-15 16:15:12,341 [INFO ]  num_workers:4
2022-08-15 16:15:12,341 [INFO ]  data_augmentation:False
2022-08-15 16:15:12,341 [INFO ]  resume:/content/drive/My Drive/VFIformer/weights/train_VF

In [33]:
import os
import glob

data_root = '/content/drive/My Drive/datasets/vimeo_triplet' 
data_list = open(os.path.join(data_root, 'tri_testlist.txt'), 'r')
flow_data = []
for item in data_list:
  name = str(item).strip()
  flow = sorted(glob.glob(os.path.join(data_root.replace('vimeo_triplet', ''), 'flows', name, '*.npy')))
  print(glob.glob(os.path.join(data_root.replace('vimeo_triplet', ''), 'flows',name, '*.npy')))
print(flow_data)

# print(glob.glob(f'/content/drive/My Drive/datasets/**/*'))

['/content/drive/My Drive/datasets/flows/00001/0001/flo21.npy', '/content/drive/My Drive/datasets/flows/00001/0001/flo23.npy']
['/content/drive/My Drive/datasets/flows/00001/0002/flo23.npy', '/content/drive/My Drive/datasets/flows/00001/0002/flo21.npy']
['/content/drive/My Drive/datasets/flows/00001/0003/flo21.npy', '/content/drive/My Drive/datasets/flows/00001/0003/flo23.npy']
['/content/drive/My Drive/datasets/flows/00001/0004/flo23.npy', '/content/drive/My Drive/datasets/flows/00001/0004/flo21.npy']
['/content/drive/My Drive/datasets/flows/00001/0005/flo21.npy', '/content/drive/My Drive/datasets/flows/00001/0005/flo23.npy']
['/content/drive/My Drive/datasets/flows/00001/0006/flo23.npy', '/content/drive/My Drive/datasets/flows/00001/0006/flo21.npy']
['/content/drive/My Drive/datasets/flows/00001/0007/flo23.npy', '/content/drive/My Drive/datasets/flows/00001/0007/flo21.npy']
['/content/drive/My Drive/datasets/flows/00001/0008/flo23.npy', '/content/drive/My Drive/datasets/flows/00001/0